  <tr>
        <td width="15%">
        </td>
        <td>
            <div align="left">
                <font size=25px>
                    <b>  Solution to Random Forest Classification Problem
                    </b>
                </font>
            </div>
        </td>
    </tr>

## Problem Statement:
The objective of the dataset is to predict the quality of wine, whether it is a good quality of wine or ordinary quality 

The data set contains several parameters which are considered important while determining the quality of a wine

The dataset consists of several predictor variables and one target variable, Quality. The target variable has values ranging from 0 to 10, the value is the rating given to each wine on a 10 point scale

## Data Definition:
 
The real-valued features are as follows:

1) **Fixed acidity:** acidity in the wine <br>
2) **Volatile acidity:** steam distillable acid in wine<br>
3) **Citric acid:** amount of citric acid present in wine<br>
4) **Residual sugar:** sugar from grapes that's left over fermentation<br>
5) **Chlorides:** amount of chloride in wine which gives a salty flavor<br>
6) **Free sulfur dioxide:** amount of sulphur dioxide which is not bound to other molecules<br>
7) **Total sulfur dioxide:** free sulphur dioxide plus amount of sulphur dioxide which is bounded to other molecules
8) **Density:** molecular density of wine<br>
9) **Ph:** pH value of wines<br>
10) **Sulphates:** amount of sulphur added to prevent oxidation of wine and bacterial spoilage<br>
11) **Alcohol:** amount of alcohol <br>

Output variable (desired target):

11) **Quality** - values ranging from 0 to 10, which gives the rating of each wine

## Content

1. **[Import Libraries](#import_lib)**
2. **[Exploratory Data Analysis](#exploratory)**
3. **[Building a RF model](#model)**

<a id='import_lib'></a>
## 1. Import Libraries

In [ ]:
# Numpy library is used to work with arrays and also has functions for working in domain of linear algebra
import numpy as np

# Pandas library provides easy to use data structures and data analysis tools
import pandas as pd

# Matplotlib library is used for plotting graphs 
import matplotlib.pyplot as plt

# Seaborn library is used for generating visualizations and is based on matplotlib
import seaborn as sns

# Sklearn pre-processing package provides several common utility functions and transformer classes for standardization of data
from sklearn import preprocessing
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Importing the library to split the data into train and test set
from sklearn.model_selection import train_test_split

# Importing the evaluation metric accuracy score of the model
from sklearn.metrics import accuracy_score

# Importing the evaluation metric confusion matrix 
from sklearn.metrics import confusion_matrix

# Importing the random forets classifier
from sklearn.ensemble import RandomForestClassifier

# Import roc_curve to compute Receiver Operating Characteristics 
from sklearn.metrics import roc_curve

# Import roc_auc_score to calculate the area uner the curve
from sklearn.metrics import roc_auc_score

# pyplot is a state-based interface to matplotlib which provides a MATLAB-like way of plotting.
from matplotlib import pyplot

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Let us read the data
wine = pd.read_csv('/content/drive/MyDrive/0.MKCE/5.Random Forest/3 Take-Home Assignment/wine.csv')

<a id='exploratory'></a>
## 2. Exploratory Data Analysis (EDA)

In [ ]:
# Ques 1 a) Visualizing the first 10 rows from the data set
wine.head(10)

In [ ]:
# Let us take a look at the data types of the variables
wine.dtypes

In [ ]:
# Ques 1 b) Check the shape of the data set
wine.shape

We can see here that the data set contains 1599 observations and 12 attributes

In [ ]:
# Ques 1 c) Show distribution of the numerical columns
sns.set()
fig = plt.figure(figsize = [15, 15])

# Specifying the columns
cols = ['fixed acidity', 'volatile acidity', 'citric acid', 
        'residual sugar', 'chlorides', 'free sulfur dioxide', 
        'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality']
count = 1

# Generating the plots
for col in cols :
    plt.subplot(4,3,count)
    sns.distplot(wine[col],hist_kws = dict(edgecolor = "k", linewidth = 1,color = 'grey'), color = 'red')
    count+=1
plt.show() 

Here we can see the distribution of all the attributes, including the target variable

In [ ]:
# Ques 1 d) Generate the correlation matrix
plt.figure(figsize = (8, 8))
sns.heatmap(wine.corr(), annot=True, linewidths=0.05, fmt= '.2f',cmap="magma")
plt.show()

A high positive correlation of 0.67 is observed between :

fixed acidity & citric acid<br>
fixed acidity & density<br>
free sulfur dioxide & total sulfur dioxide<br>
fixed acidity & pH are negatively correlated with a high absolute magnitude of 0.68 .

In [ ]:
# Ques 1 e) Visualize whether any attributes are related to the target variable
sns.set_style("whitegrid")
fig = plt.figure(figsize = [15, 15])

# Plotting the independent variables with the target variable
cols = ['fixed acidity', 'volatile acidity', 'citric acid', 
        'residual sugar', 'chlorides', 'free sulfur dioxide', 
        'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']
count = 1

# Generating the plots
for col in cols :
    plt.subplot(4,3,count)
    sns.barplot(data = wine, x = 'quality', y = col)
    count+=1
plt.show()  

Here, we can see that:
There is an increase in the following attributes as the wine quality increases :

1) citric acid <br>
2) sulphates<br>
3) alcohol<br>

A decrease is observed in the following as the wine quality increases :

1) volatile acidity<br>
2) chlorides<br>
3) pH<br>

In [ ]:
# Ques 2) Generate pair-plot for the data
sns.pairplot(wine, hue = 'quality')
plt.show()

Here we can see the histogram for each variable as well as the scatter plots based on the quality of wines

In [ ]:
# Ques 3) Generate a count plot for the target variable (quality)
sns.countplot(data = wine, x = 'quality')
plt.show()

Here we can see that most of the wines have the quality rating of 5 or 6

In [ ]:
# Ques 4) Converting the target variable 'Quality' to categorical
# Such that 
# Wines having the “Quality” value > 6.5  are assigned value 1, and 
# Wines having the “Quality” value < 6.5, are assigned value 0
# Where 0: Ordinary Quality of wine and 1: High quality of wine

wine['quality'] = wine.quality.apply(lambda x : 1 if x > 6.5 else 0)

In [ ]:
# Let us again visualize the count plot for the target variable
sns.countplot(data = wine, x = 'quality')
plt.show()

We can see that very few wines in the data set are of high quality while most of them are of ordinary quality

<a id='model'></a>
## 3. Building a RF model

In [ ]:
# Splitting the data set into train and test sets
x = wine.drop('quality',1)
y = wine['quality']

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.8, random_state = 100)

### Parameters in the classifier
1) **n_estimators:** number of trees in the forest<br>
2) **criterion:** the function to measure the quality of a split. Supported criteria are “gini” for the Gini impurity and “entropy” for the information gain<br>
3) **min_samples_split:** the minimum number of samples required to split an internal node, default = 2<br>
4) **min_samples_leaf:** the minimum number of samples required to be at a leaf node<br>
5) **max_features:** the number of features to consider when looking for the best split, default = "auto" <br>
6) **random_state:** controls both the randomness of the bootstrapping of the samples used when building trees and the sampling of the features to consider when looking for the best split at each node<br>
7) **max_depth:** the maximum depth of the tree

In [ ]:
# Ques 5) Build a random forest classifier
# Creating a random forest classifier
clf_rf = RandomForestClassifier(n_estimators = 200, criterion = 'entropy', 
                                min_samples_split = 10, min_samples_leaf = 9, max_features = "auto",
                                random_state = 500, max_depth = 12) 

In [ ]:
# Training the model 
clf_fit = clf_rf.fit(x_train, y_train) 

# Predicting the quality
y_pred = clf_fit.predict(x_test) 

In [ ]:
# Checking the accuracy of the model
accuracy_score(y_test, y_pred)

In [ ]:
# Predicting the probabilities of wine being of high quality
y_proba = clf_fit.predict_proba(x_test)

In [ ]:
# Visualizing the confusion matrix
print(confusion_matrix(y_test, y_pred))

We can see that our model is able to classify the wine quality with an accuracy of 90% and in the confusion matrix we can see that there are very few values which are being wrongly classified. Let us now take a look at the AUC for the model

In [ ]:
# Visualizing the ROC-AUC curve 

# We take the predicted values of class 1
y_predicted = y_proba[:,1]

# We check to see if the right values have been considered from the predicted values
print(y_predicted)

In [ ]:
# Using roc_curve() to generate fpr & tpr values
fpr, tpr, thresholds = roc_curve(y_test,y_predicted)

# Passing the fpr & tpr values to auc() to calculate the area under curve
from sklearn.metrics import auc
roc_auc = auc(fpr,tpr)
print("Area under the curve for first model",roc_auc)

In [ ]:
# Plotting the ROC curve
plt.figure()
plt.plot(fpr, tpr, color = 'orange', lw = 2, label = 'ROC curve (area under curve =%0.2f)'%roc_auc)

plt.plot([0,1],[0,1], color = 'darkgrey',lw = 2,linestyle='--')
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.0])
plt.xlabel('False Positive Rate (1-Specificity)')
plt.ylabel('True Positive Rate (Sensitivity)')
plt.title('ROC Curve for first model')
plt.legend(loc = "upper left")
plt.show()

So the AUC is 0.88 which is considered good for a model

### Perform Hyperparameter Tuning

In [ ]:
# Again creating a random forest classifier
clf_rf2 = RandomForestClassifier(n_estimators = 100, criterion = 'gini', 
                                min_samples_split = 2, min_samples_leaf = 5, max_features = "auto",
                                random_state = 100, max_depth = 3) 

In [ ]:
# Training the model 
clf_fit_2 = clf_rf2.fit(x_train, y_train) 

# Predicting the quality 
y_pred_2 = clf_fit_2.predict(x_test) 

In [ ]:
# Checking the accuracy of the model
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred_2)

We can see here that by changing the values of the parameter, there is a slight increase in the accuracy. Though the increase is not significant as the value only improved by 0.3, in the first model accuracy was 90.3 and now the accuracy is 90.6. Earlier we were using 'Entropy' as criterion, now we are using 'Gini' as criterion, the max depth of the tree has been decreased and so on. Similarly, we can keep on tuning the parameters, to obtain the highest accuracy

In [ ]:
# Predicting the probabilities of wine being of high quality using the second model
y_proba_2 = clf_fit.predict_proba(x_test)

In [ ]:
# Visualizing the ROC-AUC curve 

# We take the predicted values of class 1
y_predicted_2 = y_proba_2[:,1]

# Using roc_curve() to generate fpr & tpr values
fpr, tpr, thresholds = roc_curve(y_test, y_predicted_2)

# Passing the fpr & tpr values to auc() to calculate the area under curve
from sklearn.metrics import auc
roc_auc = auc(fpr,tpr)
print("Area under the curve for the second model",roc_auc)

There is no change in the AUC value of both the models